In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import folium
import requests
from matplotlib import pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from sklearn.preprocessing import StandardScaler

In [ ]:
from shapely.geometry import Point

Ouverture du fichier sur les données sociodémographiques des IRIS

In [ ]:
revenus = pd.read_csv("revenus.csv", sep=";")
revenus.head()

In [ ]:
revenus.columns = (
    revenus.columns
    .str.replace("^DISP_", "", regex=True)   # enlève le préfixe DISP_
    .str.replace("18$", "", regex=True)      # enlève le suffixe 18
    .str.lower()                             # met en minuscules
)
print(revenus.columns)

CAH
Faire un indice synthétique pour pouvoir représenter facilement les caractéristiques structurelles des quartiers sur une carte

In [ ]:
revenus.shape

In [ ]:
revenus.isna().sum()

In [ ]:
# on retire la colonne IRIS qui correspond aux identifiants, 
# la colonne DISP_TP6018 (23% de NA) et la note de précaution
rev_cah = revenus.drop(columns=["iris", "tp60", "note", "d2", "d3", "d4", "d6", "d7", "d8"])
print(rev_cah.columns)

In [ ]:
# on vérifie qu'on n'a que des valeurs numériques
rev_cah.dtypes

In [ ]:
# imputation des valeurs manquantes
for col in rev_cah.columns:
    rev_cah[col] = rev_cah[col].fillna(rev_cah[col].median())

# vérification
rev_cah.isna().sum()

In [ ]:
# normalisation
scaler = StandardScaler()
rev_scaled = scaler.fit_transform(rev_cah)

In [ ]:
# CAH
Z = linkage(rev_scaled, method='ward')

plt.figure(figsize=(12, 6))
dendrogram(Z, truncate_mode="level", p=5)
plt.title("Dendrogramme CAH")
plt.show()

In [ ]:
last = Z[:, 2]  # distances des fusions
last_rev = last[::-1]  # inversé pour l’ordre croissant

plt.figure(figsize=(10, 5))
plt.plot(range(1, 16), last_rev[:15], marker='o')
plt.xlabel("Nombre de clusters")
plt.ylabel("Distance de fusion")
plt.title("Méthode du coude (1 à 15 clusters)")
plt.grid(True)
plt.show()

In [ ]:
# 7. Découpage en clusters
clusters = fcluster(Z, 5, criterion='maxclust')
revenus["cluster"] = clusters

# Résumé
print(revenus["cluster"].value_counts())

vars_to_summarize = ["tp60", "med", "rd", "gi", "pact", "ppat", "ppsoc"]

summary = revenus.groupby("cluster")[vars_to_summarize].mean()
total = revenus[vars_to_summarize].mean()
summary_with_total = pd.concat([summary, total.to_frame().T], axis=0)
summary_with_total.index = list(summary.index) + ["Total"]

print(summary_with_total)

In [ ]:
summary = revenus.groupby("cluster").mean(numeric_only=True)
total = revenus.mean(numeric_only=True)
summary_with_total = pd.concat([summary, total.to_frame().T], axis=0)
summary_with_total.index = list(summary.index) + ["Total"]

print(summary_with_total)

In [ ]:
cluster_order = revenus.groupby("cluster")["med"].median().sort_values()
print(cluster_order)
labels = ["tres_pauvre", "pauvre", "moyen", "riche", "tres_riche"]
mapping = {cluster: labels[i] for i, cluster in enumerate(cluster_order.index)}
mapping

In [ ]:
revenus["cluster_label"] = revenus["cluster"].map(mapping)
print(revenus["cluster_label"].value_counts())

Fusionner avec les contours des IRIS

In [ ]:
gdf_iris = gpd.read_file("contours-iris-pe.gpkg")

In [ ]:
gdf_iris = gdf_iris.merge(
    revenus,
    left_on="code_iris",
    right_on="iris",
    how="left"
)
print(gdf_iris.columns)

In [ ]:
print(gdf_iris.isna().sum())
gdf_iris.shape

Données démographiques sur les IRIS

In [ ]:
population = pd.read_csv("population.csv", sep=";")
population.head()

In [ ]:
population.columns = (
    population.columns
    .str.replace("^P21_", "", regex=True)   # enlève le préfixe DISP_
    .str.replace("^C21_", "", regex=True)      # enlève le suffixe 18
    .str.lower()                             # met en minuscules
)
print(population.columns)

In [ ]:
meta = pd.read_csv("meta_population.csv", sep=";")

# garder seulement les lignes correspondant à la variable IRIS
meta_iris = meta[meta["COD_VAR"] == "IRIS"]

# ne garder que le code et le nom
meta_iris = meta_iris[["COD_MOD", "LIB_MOD"]]

In [ ]:
meta_iris["COD_MOD"] = (
    meta_iris["COD_MOD"].astype(str)
                        .apply(lambda x: x[1:] if x.startswith("0") else x)
)

In [ ]:
population["iris"] = population["iris"].astype(str)
meta_iris["COD_MOD"] = meta_iris["COD_MOD"].astype(str)

In [ ]:
population = population.merge(
    meta_iris,
    left_on="iris",
    right_on="COD_MOD",
    how="left"
)

In [ ]:
population[["iris", "LIB_MOD"]].head()

Fusion des bases de données

In [ ]:
gdf_iris = gdf_iris.merge(
    population,
    left_on="code_iris",
    right_on="COD_MOD",
    how="left"
)
print(gdf_iris.columns)

In [ ]:
print(gdf_iris[["pop", "pop_fr"]].isna().sum())
gdf_iris.shape

In [ ]:
mapping_typ_iris = {
    "H": "habitat",
    "A": "activité",
    "D": "divers",
    "Z": "autre"
}

gdf_iris["type_iris_label"] = gdf_iris["type_iris"].map(mapping_typ_iris)
gdf_iris["type_iris_label"].value_counts()

Ouverture du fichier parcoursup

In [ ]:
parcoursup = "parcoursup.csv"

df = pd.read_csv(parcoursup, sep=";")
df.head()

In [ ]:
# Normaliser toutes les colonnes
df.columns = [
    col.lower()  # tout en minuscules
       .replace(' ', '_')       # espaces → _
       .replace("'", "_")       # apostrophes → _
       .replace('(', '')        # supprimer (
       .replace(')', '')        # supprimer )
       .replace(',', '')        # supprimer ,
       .replace('.', '')        # supprimer .
       .replace('…', '')
       .replace('é','e')        # accents
       .replace('è','e')
       .replace('à','a')
       .replace('ê','e')
       .replace('ç','c')
       .replace('%','percent')
       for col in df.columns
]

# Vérifier le résultat
print(df.columns)

Relier les IRIS à parcoursup

In [ ]:
df = df.rename(columns={"coordonnees_gps_de_la_formation": "coord_gps"})

df[['latitude', 'longitude']] = df['coord_gps'].str.split(',', expand=True)
df['latitude'] = df['latitude'].astype(float)
df['longitude'] = df['longitude'].astype(float)

# Vérifier
df[['latitude','longitude']].head()

In [ ]:
df_points = gpd.GeoDataFrame(
    df,
    geometry=gpd.points_from_xy(df.longitude, df.latitude),
    crs="EPSG:4326"   # CRS WGS84 pour des coordonnées GPS
)

df_points[['geometry']].head()

In [ ]:
# Transformation des IRIS en EPSG:4326
gdf_iris = gdf_iris.to_crs(epsg=4326)

# Vérifier le CRS
print(gdf_iris.crs)

In [ ]:
df_points = df_points.set_crs(4326)
gdf_iris = gdf_iris.set_crs(4326)

df_total = gpd.sjoin(
    df_points, 
    gdf_iris[['code_iris', 'nom_iris', 'geometry', 'nom_commune', 'type_iris', "med", "rd", "ppsoc", "cluster_label", "pop", "pop1117", "pop1824", "pop15p_cs3", "pop15p_cs5", "pop15p_cs6", "pop_imm"]], 
    how="left",
    predicate="within"
)

df_total.head()

In [ ]:
print(df_total.isna()[["pop", "pop_imm", "code_iris"]].sum())
print(df_total.shape)

In [ ]:
df_total["code_iris"].value_counts()

In [ ]:
df_total.columns

In [ ]:
df_total["selectivite"].value_counts()

In [ ]:
# 1. Nombre total de formations par IRIS
total_form = (
    df_total.groupby("code_iris")
      .size()
      .reset_index(name="nb_formations")
)

# 2. Nombre de formations sélectives par IRIS
selectives = (
    df_total[df_total["selectivite"] == "formation sélective"]
    .groupby("code_iris")
    .size()
    .reset_index(name="nb_form_sel")
)

# 3. Fusion des deux résultats
result = total_form.merge(selectives, on="code_iris", how="left")

# Les IRIS sans formation sélective → 0
result["nb_form_sel"] = result["nb_form_sel"].fillna(0).astype(int)

result.head(15)

In [ ]:
for col in df_total.columns:
    if "taux" in col.lower():
        print(col)

In [ ]:
df_total["taux_d’acces"].quantile([0.25, 0.333, 0.5, 0.75])

In [ ]:
(df_total["taux_d’acces"] < 50).mean()

In [ ]:
df_total["percent_d’admis_neo_bacheliers_boursiers"].quantile([0.25, 0.333, 0.5, 0.667, 0.75])

In [ ]:
(df_total["percent_d’admis_neo_bacheliers_boursiers"] < 30).mean()

In [ ]:
# créer les colonnes pour les formations très sélectives et avec un haut taux de boursiers
df_total["tres_select"] = df_total["taux_d’acces"] < 50
df_total["bcp_boursiers"] = df_total["percent_d’admis_neo_bacheliers_boursiers"] > 30

# compter par IRIS
result2 = df_total.groupby("code_iris")[["tres_select", "bcp_boursiers"]].sum().reset_index()

# fusionner aux autres colonnes créées 
result3 = result.merge(result2, on="code_iris", how="left")

# afficher
result3.head()

Ajouter ces colonnes à la base sur les iris

In [ ]:
gdf_iris = gdf_iris.merge(result3, on="code_iris", how="left")
gdf_iris[["nb_formations", "nb_form_sel", "tres_select", "bcp_boursiers"]].head(15)

In [ ]:
# Remplacer les NaN par 0 pour les colonnes issues des données parcoursup
gdf_iris["nb_formations"] = gdf_iris["nb_formations"].fillna(0)

# Vérification
gdf_iris["nb_formations"].head(30)

In [ ]:
gdf_iris["nb_formations_cat"] = "0"

# sélectionner les valeurs > 0
mask = gdf_iris["nb_formations"] > 0

# créer les quantiles sur le reste
gdf_iris.loc[mask, "nb_formations_cat"] = pd.qcut(gdf_iris.loc[mask, "nb_formations"], 
                                            q=2, 
                                            labels=["Q1","Q2"])

gdf_iris.groupby("nb_formations_cat")["pop"].mean()

In [ ]:
# Centrer la carte sur Besançon
besancon_lon, besancon_lat = 6.025, 47.237
#m = folium.Map(location=[besancon_lat, besancon_lon], zoom_start=13)

# # Ajouter les polygones IRIS
# folium.GeoJson(
#     gdf_iris,
#     name="IRIS",
#     style_function=lambda x: {"fillColor": "blue", "color": "black", "weight": 1, "fillOpacity": 0.2}
# ).add_to(m)

# Filtrer les points valides
#df_points_valid = df_points.dropna(subset=['latitude', 'longitude'])

# Boucle sur les points valides
# for idx, row in df_points_valid.iterrows():
#     folium.CircleMarker(
#         location=[row['latitude'], row['longitude']],
#         radius=3,
#         color="red",
#         fill=True,
#         fill_opacity=0.7
#     ).add_to(m)

# Ajouter contrôle des couches
#folium.LayerControl().add_to(m)

# Afficher la carte
#m
print("ok")

In [ ]:
# 1) Ajouter un code département à partir du code IRIS
gdf_iris["code_iris"] = gdf_iris["code_iris"].astype(str)
gdf_iris["code_dept"] = gdf_iris["code_iris"].str[:2]

# 2) Garder uniquement l'Île-de-France
idf_deps = ["75", "77", "78", "91", "92", "93", "94", "95"]
gdf_idf = gdf_iris[gdf_iris["code_dept"].isin(idf_deps)].copy()

print("Nombre d'IRIS en IDF :", gdf_idf.shape[0])

# 3) Palette de couleurs pour les types de quartiers (clusters)
cluster_colors = {
    "tres_pauvre": "#b30000",  # rouge foncé
    "pauvre":      "#fc8d59",  # orange
    "moyen":       "#fee08b",  # jaune
    "riche":       "#91bfdb",  # bleu clair
    "tres_riche":  "#4575b4",  # bleu foncé
}

def style_cluster(feature):
    label = feature["properties"].get("cluster_label")
    color = cluster_colors.get(label, "#cccccc")  # gris si NaN
    return {
        "fillColor": color,
        "color": "black",
        "weight": 0.3,
        "fillOpacity": 0.6,
    }

# 4) Filtrer les formations qui sont dans un IRIS IDF
idf_iris_codes = set(gdf_idf["code_iris"].astype(str).unique())

df_points_idf = df_total[
    df_total["code_iris"].astype(str).isin(idf_iris_codes)
].dropna(subset=["latitude", "longitude"])

print("Nombre de formations en IDF :", df_points_idf.shape[0])

# 5) Créer une carte centrée sur Paris
m = folium.Map(
    location=[48.8566, 2.3522],
    zoom_start=10,
    max_zoom=10,
    min_zoom=10,
    dragging=False,
    scrollWheelZoom=False,
    doubleClickZoom=False,
    zoomControl=False
)


# 6) Ajouter les polygones IRIS colorés selon le type de quartier
folium.GeoJson(
    gdf_idf,
    name="Quartiers (IRIS)",
    style_function=style_cluster,
    tooltip=folium.GeoJsonTooltip(
        fields=["nom_iris", "nom_commune", "cluster_label"],
        aliases=["IRIS", "Commune", "Type de quartier"],
        localize=True
    ),
).add_to(m)

# 7) Ajouter les formations en points rouges
for _, row in df_points_idf.iterrows():
    folium.CircleMarker(
        location=[row["latitude"], row["longitude"]],
        radius=1,
        color="red",
        fill=True,
        fill_opacity=0.8,
    ).add_to(m)

folium.LayerControl().add_to(m)
#m
print("ok")